## EU JRC global flood depth-damage functions

### CITATION
Huizinga, J., De Moel, H. and Szewczyk, W., Global flood depth-damage functions: Methodology and the database with guidelines, EUR 28552 EN, Publications Office of the European Union, Luxembourg, 2017, ISBN 978-92-79-67781-6, doi:10.2760/16510, JRC105688.

### LINK
https://publications.jrc.ec.europa.eu/repository/handle/JRC105688

### NOTES
Data from copy_of_global_flood_depth-damage_functions__30102017.xlsx converted into CSV format in raw.csv. Ingested into vulnerability model as standard json.


In [1]:
import json
import numpy as np
import pandas as pd

df = pd.read_csv("raw.csv")

# consistent with physrisk continent definition
location_mappings = {
    "Europe": "Europe",
    "North America": "North America",
    "Central & South America": "South America",
    "Asia": "Asia",
    "Africa": "Africa",
    "Oceania": "Oceania",
    "Global": "Global",
}
type_mappings = {
    "Residential buildings": "Buildings/Residential",
    "Commercial buildings": "Buildings/Commercial",
    "Industrial buildings": "Buildings/Industrial",
}

data = {"items": []}

curve_list = data["items"]
for mapping in type_mappings:
    type_df = df[df["Type"] == mapping]
    flood_depth = type_df["Flood depth [m]"].to_numpy()
    for location in location_mappings:
        # whether zero depth is considered really zero or a flood event with smallest depth
        zero_as_minimum = True if location == "North America" else False
        # for North America, the 0 depth damage is for flooding of any depth. We consider that a 1 cm inundation.
        depth = np.concatenate([[0, 0.01], flood_depth[1:]]) if zero_as_minimum else flood_depth

        mean = type_df[location + "_Mean"].to_numpy()
        std = type_df[location + "_Std"].to_numpy()
        mean = np.concatenate([[0], mean]) if zero_as_minimum else mean
        std = np.concatenate([[0], std]) if zero_as_minimum else std
        if np.any(np.isnan(mean)):
            mean = []
        if np.any(np.isnan(std)):
            std = []
        curve_list.append(
            {
                "asset_type": type_mappings[mapping],
                "event_type": "RiverineInundation",
                "location": location_mappings[location],
                "impact_type": "Damage",
                "intensity": list(depth),
                "intensity_units": "m",
                "impact_mean": list(mean),
                "impact_std": list(std),
            }
        )

with open("EU JRC global flood depth-damage functions.json", "w") as f:
    vulnerability_json = json.dumps(data, sort_keys=True, indent=4)
    f.write(vulnerability_json)